ImmoScoutScraper
go to immobilienscout24.de, do your search, specifying all search criteria, copy url and paste into input cell below.
Specify SaveFolder to have the result .csv saved there

Further steps
Expose-ID will be read and Immobilien-Scout Expose-API will be called

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
import os.path
from os import path

LinkList=[]
PageNo, ResultPageNo = 0, 1

In [2]:
#Inputs
#Paste Search URL (with filters encoded into it) here
URLwCriteria = "https://www.immobilienscout24.de/Suche/radius/wohnung-kaufen?centerofsearchaddress=Landshut%20(Kreis);;;1276002051;Bayern;&numberofrooms=1.0-&price=-950000.0&livingspace=40.0-&geocoordinates=48.54481;12.19322;10.0&enteredFrom=one_step_search"

#specify folder to save results in (relative to JupyterProjects folder)
#SaveFolder = "JupyterProjects/ImmoScout/"
DownloadFileName = "ExposeURLs_WhngKaufen_LA" #date and .csv will be added automaticalle. recommendation: specify Object type and location
DownloadMasterName = "Master_WhngKaufen_LA"

In [3]:
#preprocess input
##remove the "enteredFrom=XYZ" and add add "pagenumber=" from URL (assumption: user does not go to other search pages before copying URL)
if "enteredFrom=" in URLwCriteria:
    URLwCriteria = URLwCriteria[:URLwCriteria.find("enteredFrom=",32)]
URLwCriteria += "pagenumber="
##construct paths from filenames and folder

DateTimeToday = pd.to_datetime("today")
DateToday = str(pd.datetime.now().date())
DownloadFilePath = DateToday+"_"+DownloadFileName+".csv" #evtl. = SaveFolder+...
DownloadMasterPath = DownloadMasterName+".csv" #evtl. = SaveFolder+...

In [4]:
#Loop over result pages. starting with 1 up number of results divided by 20 (which is the number of results displayed per result page)

while PageNo < ResultPageNo:
    PageNo += 1

    URLwPageNo = URLwCriteria + str(PageNo)
    page = requests.get(URLwPageNo, timeout=3)
    soup = BeautifulSoup(page.content, 'html.parser')

    #find number of search results to calculate number of pages to check, if on first page
    if PageNo == 1:
        titletext = soup.find ("div", class_="palm-hide margin-bottom-m")
        #print(titletext)
        resultnumberhtml = titletext.find("span")
        resultnumber=resultnumberhtml.text
        #print(resultnumber)
        #calculate number of pages to check
        ResultPageNo = math.ceil(int(resultnumber)/20)
        #print(ResultPageNo)

    #Extract Links from html and then extract only text after "href"
    HtmlLinks = soup.find_all('a')
    #print(HtmlLinks)
    for HtmlLink in HtmlLinks:
        LinkText = HtmlLink.get("href")
        #print(LinkText)
        LinkList.append(LinkText[:17])  

In [5]:
#print(LinkList)
#clean up
del soup

In [6]:
#Create list of URLs of (only) Exposes, extracted from list of all links after looping over result pages
ExposeURLList = list(set(["https://www.immobilienscout24.de"+ExposePath for ExposePath in LinkList if "/expose/" in ExposePath])) #use set to remove duplicates. order will be lost
#print(ExposeURLList)
#print(len(ExposeURLList))

#Create list of Expose IDs from ExposeURLList
ExposeIDList = [ExposeURL[40:49] for ExposeURL in ExposeURLList]
#print(ExposeIDList)

In [7]:
#Create Pandas DataFrame with ExposeID, ExposeURL and Download date
ExposeDF = pd.DataFrame({"ExposeID":ExposeIDList, "ExposeURL":ExposeURLList})
ExposeDF["DownloadDate"] = DateTimeToday
ExposeDF["SearchURL"] = URLwCriteria+str(1)
ExposeDF.head()

ExposeID                                          ExposeURL  \
0  114892178  https://www.immobilienscout24.de/expose/114892178   
1  115435502  https://www.immobilienscout24.de/expose/115435502   
2  110787901  https://www.immobilienscout24.de/expose/110787901   
3  113716154  https://www.immobilienscout24.de/expose/113716154   
4  114929953  https://www.immobilienscout24.de/expose/114929953   

                DownloadDate  \
0 2020-01-26 22:09:45.714092   
1 2020-01-26 22:09:45.714092   
2 2020-01-26 22:09:45.714092   
3 2020-01-26 22:09:45.714092   
4 2020-01-26 22:09:45.714092   

                                           SearchURL  
0  https://www.immobilienscout24.de/Suche/radius/...  
1  https://www.immobilienscout24.de/Suche/radius/...  
2  https://www.immobilienscout24.de/Suche/radius/...  
3  https://www.immobilienscout24.de/Suche/radius/...  
4  https://www.immobilienscout24.de/Suche/radius/...

In [8]:
#save DataFrame as .csv with current date to specified folder
ExposeDF.to_csv(DownloadFilePath, index=False)

In [9]:
#append downloads to master file which collects all the downloaded URLs
##check if master file exists. if yes, do not add headers to file, if no add header
if path.exists(DownloadMasterPath):
    ExposeDF.to_csv(DownloadMasterPath, mode = "a", header = False)
else:
    ExposeDF.to_csv(DownloadMasterPath, mode = "a", header = True)

In [10]:
#open master file, write current entries (date) to "New"-file, if same Expose-ID not in List yet (with older date)
MasterDF = pd.read_csv(DownloadMasterPath, index_col=0)
MasterDF.head()

#check master for duplicate ExposeIDs, get their Indices and create a new DF from ExposeID with duplicates dropped
#or check master for duplicate ExposeIDs and delete latest ones (i.e. keep first one and delete appended one), filter for current date
idx = MasterDF.ExposeID
idx = idx.drop_duplicates(keep = "first")
#print(idx)

In [11]:
NewDF = MasterDF.loc[MasterDF.ExposeID.isin(idx)]
NewDF.shape

(126, 4)

In [12]:
#open "New"-file and use Expose-API to add information from Exposes

In [13]:
#save IDs + information in final file

instruction to open result file (.csv) in Excel (or similar):
1) open .csv file in Excel
2) mark first column ("A")
3) go to "Data" -> "Text to columns"
4) choose "delimited", then "comma" as separator. you can choose formats for certain columns but not necessary
5) save copy as Excel-File. Do (beeter) not save changes to the original file, if you still want to process them

In [14]:
#Issues
#1) removal of "other offers". probably keep in URLs (should be about 10 suggestions per search, only on last page) then check location in Expose
# or don t use "set" for ExposeURLList to keep the order and then cut the last 10 
#2) only checks, if Expose-ID has been downloaded before. If Expose has been updated (e.g. price), it will still be ignored. Possible workaround: download all Exposes and check, if there s a date for last update in API-response (JSON)
#3) use github to track issues and for versioning

#Features to be added: 
#a) in progress: compare to "old" masterfile to also create a list of only new ones
#c) use shapes instead of cities in search. should be encoded in URLs of saved searches so probably not much of a change
#d) upload resultfile to nextcloud
#e) cronjob on raspi